In [342]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
import nltk 
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
# fix random seed for reproducibility
np.random.seed(7)
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')
from nltk.tokenize import sent_tokenize, word_tokenize

In [343]:
# read the data and store data in DataFrame titled news_data
news_data = pd.read_csv("News_Dataset.csv", header=None, skiprows=1, encoding='utf-8') 
# print a summary of the data in news_data
news_data[1] = news_data[1].replace(['1-Septic', '0-Pure'], [1, 0])
news_data[0] = news_data[0].replace('\n', '', regex=True).str.strip()
news_data.head()

,0,1
0,While the Congress seems to be in revival mode...,1
1,"In Congress, it’s a problem of plenty with too...",1
2,Though the BJP leadership has been reportedly ...,0
3,The political predicament also appears to be t...,0
4,"The party leadership, mainly PM Narendra Modi ...",1


In [344]:
news_data.dropna(axis=0, inplace=True)
x = news_data[0]
y = news_data[1]
x_train , x_test ,y_train ,  y_test = train_test_split(x, y, test_size = 0.2)


In [345]:
tokenizer = Tokenizer(num_words=5000) #adding this parameter can  responsible for setting the size of the vocabulary i.e the most common num_words
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [346]:
maxlen = 100
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)
print(type(x_train))
# vocab_size = len(tokenizer.word_index) + 1  

# data = []
# punc = '.'
# for i in x_train:
#     i = i[:-1]
#     token = word_tokenize(i)
#     if punc in token:
#         for index ,val in enumerate(token):
#             if punc == val:
#                 token.pop(index) 
# #      x_train = tokenizer.texts_to_sequences(token)
# #    print(type(token))
# #    token=np.array(token)
#     data.append(token)
# #     ' '.join(map(data,token ))
# x_train = tokenizer.texts_to_sequences(data) 
# x_test = tokenizer.texts_to_sequences(x_test)
# x_train=np.ndarray(x_train)
# x_test=np.ndarray(x_test)
# news_data.info()
print(x_train)

<class 'numpy.ndarray'>
[[   1  377 1068 ...    0    0    0]
 [  78  169    1 ...    0    0    0]
 [   1  791    9 ...    0    0    0]
 ...
 [ 281    3  152 ...    0    0    0]
 [   1 1415   29 ...    0    0    0]
 [  21    9  682 ...    0    0    0]]


In [347]:
def RNN():
    inputs = Input(name='inputs',shape=[maxlen])
    layer = Embedding(5000,50,input_length=maxlen)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model


In [348]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

Model: "functional_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 100)]             0         
_________________________________________________________________
embedding_27 (Embedding)     (None, 100, 50)           250000    
_________________________________________________________________
lstm_27 (LSTM)               (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_54 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)               

In [349]:
model.fit(x_train,y_train,batch_size=128,epochs=20)

Epoch 1/20
12/12 [==============================] - 1s 77ms/step - loss: 0.6446 - accuracy: 0.7500
Epoch 2/20
12/12 [==============================] - 1s 77ms/step - loss: 0.5269 - accuracy: 0.7993
Epoch 3/20
12/12 [==============================] - 1s 76ms/step - loss: 0.5116 - accuracy: 0.7993
Epoch 4/20
12/12 [==============================] - 1s 113ms/step - loss: 0.5016 - accuracy: 0.7993
Epoch 5/20
12/12 [==============================] - 1s 101ms/step - loss: 0.5062 - accuracy: 0.7993
Epoch 6/20
12/12 [==============================] - 1s 92ms/step - loss: 0.5055 - accuracy: 0.7993
Epoch 7/20
12/12 [==============================] - 1s 84ms/step - loss: 0.5068 - accuracy: 0.7993
Epoch 8/20
12/12 [==============================] - 1s 78ms/step - loss: 0.5043 - accuracy: 0.7993
Epoch 9/20
12/12 [==============================] - 1s 79ms/step - loss: 0.5054 - accuracy: 0.7993
Epoch 10/20
12/12 [==============================] - 1s 77ms/step - loss: 0.5045 - accuracy: 0.7993
Epoch 1

In [350]:
accr = model.evaluate(x_test,y_test)

12/12 [==============================] - 0s 11ms/step - loss: 0.5304 - accuracy: 0.7778


In [351]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))



Test set
  Loss: 0.530
  Accuracy: 0.778


In [352]:
def mapper(a):
    a = tokenizer.texts_to_sequences(a)
    return pad_sequences(a, padding='post', maxlen=maxlen)


x_eval = mapper(['he said, "the world goes round and round and round"',
        'he said, "the world has changed devastatingly"'])

y_eval = np.array([0, 0])
model.evaluate(x_eval, y_eval)

for i in model.predict(x_eval):
    print(round(i[0]))

1/1 [==============================] - 0s 830us/step - loss: 0.2322 - accuracy: 1.0000
0.0
0.0


In [353]:
from sklearn.metrics import confusion_matrix

y_eval = model.predict(x_test)
y_eval = [round(i[0]) for i in y_eval]
confusion_matrix(y_test, y_eval)

array([[280,   0],
       [ 80,   0]])